In [22]:
import numpy as np
import matplotlib.pyplot as plt

In [23]:
H0, Omega_M, Omega_lambda, w, M = np.loadtxt('estimate_combined.txt', unpack=True)
H0_SNIa, Omega_M_SNIa, Omega_lambda_SNIa, w_SNIa, M_SNIa = np.loadtxt('estimate_SNIa.txt', unpack=True)
H0_GRB, Omega_M_GRB, Omega_lambda_GRB, w_GRB, M_GRB = np.loadtxt('estimate_GRB.txt', unpack=True)
H0_clusters, Omega_M_clusters, Omega_lambda_clusters, w_clusters, M_clusters = np.loadtxt('estimate_clusters.txt', unpack=True)

In [27]:
h = np.quantile(H0, 0.16)
print(h)

65.61957906338503
